In [1]:
# Importing dependecies

import numpy as np      # For matrix math
import tensorflow as tf # For ML
import helpers          # formatting data and generating random seq of data

tf.reset_default_graph()
sess = tf.InteractiveSession()

## Goals
Here we want to learn how memory works in sequence 2 sequence models. Here given a sequence of inputs we want to encode it and then decode it to have the same sequence.

## Architecture
Bidirectional encoder, to have the full context of future and past.

In [2]:
tf.__version__

'1.0.1'

In [3]:
PAD = 0
EOS = 1

vocab_size = 10
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = 2*encoder_hidden_units
# double because we want our output values to be little different
# so that we can minimize the errors

In [4]:
# input placeholders: gateways for our computational graph
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_input_length')
decoder_targets = tf.placeholder(shape=(None,None), dtype=tf.int32, name='decoder_targets')

In [5]:
# embeddings
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1),
                         dtype=tf.float32)
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [6]:
# define encoder
with tf.variable_scope('encoder_cell', reuse=True): 
    encoder_cell = tf.contrib.rnn.BasicLSTMCell(encoder_hidden_units)

In [7]:
((encoder_fw_outputs, encoder_bw_outputs),
 (encoder_fw_final_state, encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_bw=encoder_cell,
                                                                                     cell_fw=encoder_cell,
                                                                                     inputs=encoder_inputs_embedded,
                                                                                     sequence_length=encoder_inputs_length,
                                                                                     dtype=tf.float32, time_major=True)
                                                                                    )

In [9]:
encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

#TF Tuple used by LSTM Cells for state_size, zero_state, and output state.
encoder_final_state = tf.contrib.rnn.LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

## Decoder

In [11]:
decoder_cell = tf.contrib.rnn.BasicLSTMCell(decoder_hidden_units)
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))
decoder_lengths = encoder_inputs_length + 3